# Natural Language Processing : Classic to Deep Methods for Sentiment Analysis

## Resources

Bag-Of-Word and TF-IDF:

https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/

Recurrent Neural Networks (RNNs):

https://medium.com/towards-data-science/illustrated-guide-to-recurrent-neural-networks-79e5eb8049c9

Long Short Term Memory networks (LSTMs):

https://medium.com/towards-data-science/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

http://colah.github.io/posts/2015-08-Understanding-LSTMs/

Word embeddings:

http://jalammar.github.io/illustrated-word2vec/

## Import Modules

In [2]:
import os
import numpy as np
import pandas as pd
import re

from collections import defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.lm.vocabulary import Vocabulary

nltk.download('stopwords')

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/thibault.launois@Digital-
[nltk_data]     Grenoble.local/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Today we are going to tackle the sentiment analysis problem, a *text classification* problem. The idea is pretty simple : we want to automatically predict whether a text expresses positive or negative sentiments. To do so we will use the IMDB dataset, that contains 50000 movie reviews from the www.imdb.com website, and their corresponding sentiment : positive or negative. It is thus a binary classification problem, where we want to predict a binary target $y \in \{0,1\}$. We will go through different ways of encoding a text in a vectorial form $x \in \mathbb{R}^d$, as well as different classification models, from classic ways to modern deep learning models.

## Load the dataset

Load the dataset and explore a bit the data :

In [3]:
#Load and print the dataset
imdb_dataset_original=pd.read_csv('../data/IMDB Dataset.csv')
imdb_dataset = imdb_dataset_original.copy()
imdb_dataset.head(10)

,review,sentiment
0,I saw this movie last night after waiting ages...,positive
1,"This was, so far, the worst movie I have seen ...",negative
2,"In a time of magic, barbarians and demons abou...",positive
3,I had high expectations of this movie (the tit...,negative
4,This is a film of immense appeal to a relative...,negative
5,An hilariously accurate caricature of trying t...,positive
6,I watch most movies that Nick Mancuso is in be...,positive
7,This is one of those films that's more interes...,negative
8,A wonderful and gritty war film that focuses o...,positive
9,"Man, some of you people have got to chill. Thi...",positive


In [4]:
#Print first review:
print(imdb_dataset["review"][10])

Wow. I don't even really remember that much about this movie, except that it stunk.<br /><br />The plot's basically; a girl's parents neglect her, so this sicko PokeMon pretends to be her dad. Am I the only one disturbed by that? Then, this weirdo PokeMon kidnaps Ash's mom to pretend to be the girl's. I don't care if he was trying to make the girl happy, that's just gross.<br /><br />There was no real plot. The girl was just a whiny brat who wanted things her own way. She played with Unowns, was the "daughter" of Entei and apparently could grow and shrink in age on a whim with the help of her "dad".<br /><br />That's pretty much all I can remember, but I think you can take it as a hint, and not see it. (Or if you do see it, don't expect much.) 1 out of 10.<br /><br />Seriously. If you want a PokeMon movie, rent "PokeMon; the First Movie".


In [5]:
#Print the two classes size
imdb_dataset['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

## Text preprocessing

As you can see the text is quite messy, and before encoding our text into features, we are going to go through different preprocessing steps in order to clean it:
* Removing the HTML tags.
* Removing other special characters : this means all non alphanumeric characters, including punctuation.
* Lowercase the text.
* Tokenization : split a text as a list of words now called tokens.
* Stemming : removing all the suffixes from conjugation, plural, ... In order to bring a word back to its root form. For example.
* Removing stopwords : words like 'to', 'a', 'the', ... are called stopwords, we remove them as they are too frequent words and generally just add noise.

Fill the following functions to perform each of these steps. You are free to use the libraries of your choice to do so. Try to not reinvent the wheel!

In [6]:
def remove_html_tags(text):
    """
    Input: str : A string to clean from html tags
    Output: str : The same string with html tags removed
    """
    cleaner = re.compile('<.*?>')
    clean_text = re.sub(cleaner, ' ', text)
    return clean_text

In [17]:
imdb_dataset['review']=imdb_dataset['review'].apply(remove_html_tags)
imdb_dataset['review'][10]

'Wow. I don\'t even really remember that much about this movie, except that it stunk.  The plot\'s basically; a girl\'s parents neglect her, so this sicko PokeMon pretends to be her dad. Am I the only one disturbed by that? Then, this weirdo PokeMon kidnaps Ash\'s mom to pretend to be the girl\'s. I don\'t care if he was trying to make the girl happy, that\'s just gross.  There was no real plot. The girl was just a whiny brat who wanted things her own way. She played with Unowns, was the "daughter" of Entei and apparently could grow and shrink in age on a whim with the help of her "dad".  That\'s pretty much all I can remember, but I think you can take it as a hint, and not see it. (Or if you do see it, don\'t expect much.) 1 out of 10.  Seriously. If you want a PokeMon movie, rent "PokeMon; the First Movie".'

In [7]:
def remove_special_characters(text):
    """
    Input: str : A string to clean from non alphanumeric characters
    Output: str : The same strings without non alphanumeric characters
    """
    cleaner = re.compile("[^a-zA-Z0-9]")
    clean_text = re.sub(cleaner, ' ', text)
    return clean_text

In [19]:
imdb_dataset['review']=imdb_dataset['review'].apply(remove_special_characters)
imdb_dataset['review'][10]

'Wow  I don t even really remember that much about this movie  except that it stunk   The plot s basically  a girl s parents neglect her  so this sicko PokeMon pretends to be her dad  Am I the only one disturbed by that  Then  this weirdo PokeMon kidnaps Ash s mom to pretend to be the girl s  I don t care if he was trying to make the girl happy  that s just gross   There was no real plot  The girl was just a whiny brat who wanted things her own way  She played with Unowns  was the  daughter  of Entei and apparently could grow and shrink in age on a whim with the help of her  dad    That s pretty much all I can remember  but I think you can take it as a hint  and not see it   Or if you do see it  don t expect much   1 out of 10   Seriously  If you want a PokeMon movie  rent  PokeMon  the First Movie  '

In [8]:
def lowercase_text(text):
    """
    Input: str : A string to lowercase
    Output: str : The same string lowercased
    """
    return text.lower()

In [28]:
imdb_dataset['review']=imdb_dataset['review'].apply(lowercase_text)
imdb_dataset['review'][10]

'wow  i don t even really remember that much about this movie  except that it stunk   the plot s basically  a girl s parents neglect her  so this sicko pokemon pretends to be her dad  am i the only one disturbed by that  then  this weirdo pokemon kidnaps ash s mom to pretend to be the girl s  i don t care if he was trying to make the girl happy  that s just gross   there was no real plot  the girl was just a whiny brat who wanted things her own way  she played with unowns  was the  daughter  of entei and apparently could grow and shrink in age on a whim with the help of her  dad    that s pretty much all i can remember  but i think you can take it as a hint  and not see it   or if you do see it  don t expect much   1 out of 10   seriously  if you want a pokemon movie  rent  pokemon  the first movie  '

In [9]:
def tokenize_words(text):
    """
    Input: str : A string to tokenize
    Output: list of str : A list of the tokens splitted from the input string
    """
    return re.split(r"\s+", text.strip())

In [10]:
imdb_dataset['review']=imdb_dataset['review'].apply(tokenize_words)
imdb_dataset.head(10)

,review,sentiment
0,"[I, saw, this, movie, last, night, after, wait...",positive
1,"[This, was,, so, far,, the, worst, movie, I, h...",negative
2,"[In, a, time, of, magic,, barbarians, and, dem...",positive
3,"[I, had, high, expectations, of, this, movie, ...",negative
4,"[This, is, a, film, of, immense, appeal, to, a...",negative
5,"[An, hilariously, accurate, caricature, of, tr...",positive
6,"[I, watch, most, movies, that, Nick, Mancuso, ...",positive
7,"[This, is, one, of, those, films, that's, more...",negative
8,"[A, wonderful, and, gritty, war, film, that, f...",positive
9,"[Man,, some, of, you, people, have, got, to, c...",positive


In [11]:
def remove_stopwords(token_list):
    """
    Input: list of str : A list of tokens
    Output: list of str : The new list with removed stopwords tokens
    """
    stop_words = set(stopwords.words('english'))
    filtered_list = [word for word in token_list if word not in stop_words]
    return filtered_list

In [40]:
imdb_dataset['review']=imdb_dataset['review'].apply(remove_stopwords)
imdb_dataset.head(10)

,review,sentiment
0,"[saw, movie, last, night, waiting, ages, ages,...",positive
1,"[far, worst, movie, seen, entire, life, seen, ...",negative
2,"[time, magic, barbarians, demons, abound, diab...",positive
3,"[high, expectations, movie, title, translated,...",negative
4,"[film, immense, appeal, relatively, well, defi...",negative
5,"[hilariously, accurate, caricature, trying, se...",positive
6,"[watch, movies, nick, mancuso, frankly, love, ...",positive
7,"[one, films, interesting, watch, academic, per...",negative
8,"[wonderful, gritty, war, film, focuses, inner,...",positive
9,"[man, people, got, chill, movie, artistic, gen...",positive


In [12]:
def stem_words(token_list):
    """
    Input: list of str : A list of tokens to stem
    Output: list of str : The list of stemmed tokens
    """
    ps = PorterStemmer()
    stemmed_list = [ps.stem(word) for word in token_list]
    return stemmed_list

In [44]:
imdb_dataset['review']=imdb_dataset['review'].apply(stem_words)
imdb_dataset.head(10)

,review,sentiment
0,"[saw, movi, last, night, wait, age, age, relea...",positive
1,"[far, worst, movi, seen, entir, life, seen, re...",negative
2,"[time, magic, barbarian, demon, abound, diabol...",positive
3,"[high, expect, movi, titl, translat, get, rid,...",negative
4,"[film, immens, appeal, rel, well, defin, group...",negative
5,"[hilari, accur, caricatur, tri, sell, script, ...",positive
6,"[watch, movi, nick, mancuso, frankli, love, gu...",positive
7,"[one, film, interest, watch, academ, perspect,...",negative
8,"[wonder, gritti, war, film, focus, inner, torm...",positive
9,"[man, peopl, got, chill, movi, artist, geniu, ...",positive


Let's join all that together and apply it to our dataset. The following function simply chains all the preprocessing steps you just implemented. 

It adds the `list_output` flag, if False it will reconcatenate all the preprocessed tokens into a single string (with spaces between tokens), if True it will keep each sentence as a list of tokens. Depending on the libraries you will use for the next steps, it can be useful to have one or the other representation.

In [13]:
def normalize_text_dataset(
        dataset, text_col_name = 'review', html_tags = True,
        special_chars = True, lowercase = True , stemming = True , 
        stopwords = True, list_output = False ):
    """
    Apply the choosen preprocessing steps to a corpus of texts and return the 
    preprocessed corpus. The list_output flag allows to return either a list
    of token, or a rejoined string with spaces between the preprocessed tokens.
    """
    def rejoin_text(token_list):
        return ' '.join(token_list)
    
    output = dataset.copy()
    
    if html_tags : 
        output[text_col_name] = output[text_col_name].apply(remove_html_tags)
        
    if special_chars :
        output[text_col_name] = output[text_col_name].apply(remove_special_characters)
        
    if lowercase :
        output[text_col_name] = output[text_col_name].apply(lowercase_text)
    
    #Tokenization for next steps:
    output[text_col_name] = output[text_col_name].apply(tokenize_words)
    
    if stopwords :
        output[text_col_name] = output[text_col_name].apply(remove_stopwords)
        
    if stemming :
        output[text_col_name] = output[text_col_name].apply(stem_words)
        
    if not list_output :
        output[text_col_name] = output[text_col_name].apply(rejoin_text)
        
    return output

In [133]:
imdb_clean_dataset = normalize_text_dataset(
    imdb_dataset_original, html_tags = True,
    special_chars = True, lowercase = True , stemming = True , 
    stopwords = True, list_output = False)

## Text classification with Bag-Of-Words

Now we have cleaned the reviews of our dataset, how do we represent them as vectors in order to classify it ? 
One classic way to achieve that is the Bag-Of-Words (BOW) approach. To encode a text in a bag of word, we first need to know all the different words $w$ that appear in all our reviews, called the vocabulary : $w \in \mathcal{V}$. For each word $w$ we attribute an index $idx(w) = i$ with $i \in \{0, |\mathcal{V}|-1\}$, and represent a review $r$ as a vector of the size of the vocabulary $x_r \in \mathbb{R}^{|\mathcal{V}|}$. To encode a review we are simply going to count how many time each word appears and assign it at its corresponding index in the bag-of-words vector : $x_{r,i} = count(w,r)$, where i = idx(w). 

This means that we completely disregard the words order, and simply take into account the number of times each word appears in each review to represent them. There are many variations of this concept, TF-IDF (term frequency-inverse document frequency) for example, gives more weight to uncommon words. Read more about BOW and TF-IDF there:

https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/

Let's start with bag-of-words. In general we don't consider the whole vocabulary but only some of the most frequent words in order to reduce the dimensionality and avoid noise from rare words. Here we will only consider the 10 000 most frequent words of the training set, meaning the words that are only in the test set will be ignored. Thus we have : $x_r \in \mathbb{R^{10000}}$.

Encode all the reviews as bag-of-words, and train and evaluate a logistic regression model on the following train test splits. As we have seen previously, if we wanted to investigate this model we should also grid search for hyperparameters by doing a cross-validation with validation sets, etc. However this is not the goal today, so we'll simply go for a train/test split for this experiment. Concerning the evaluation metrics, in this case we care equally about correctly predicting the positives and the negatives, and we have a balanced dataset, thus we can simply use accuracy this time.

Once again, don't do everything from scratch and try to find libraries that propose implementations of these concepts !

In [145]:
max_vocab_size = 10000 

#Train/test split:

lb = LabelBinarizer()
sentiment_labels = lb.fit_transform(imdb_clean_dataset['sentiment'])

train_reviews = imdb_clean_dataset.review[:25000]
test_reviews = imdb_clean_dataset.review[25000:]

train_sentiments = sentiment_labels[:25000].ravel()
test_sentiments = sentiment_labels[25000:].ravel()

In [140]:
# bag of word
# 10 000 most frequent words
vectorizer = CountVectorizer(
    input='content',
    lowercase=False,
    
    max_features=max_vocab_size
    )

x_train = vectorizer.fit_transform(train_reviews)
x_test = vectorizer.transform(test_reviews)

In [ ]:
# train logistic regression
model = LogisticRegression(max_iter=10000)

model.fit(x_train, train_sentiments)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,10000
,multi_class,'deprecated'


In [159]:
# evaluate with accuracy
y_pred = model.predict(x_test)

accuracy = accuracy_score(test_sentiments, y_pred)
print(f"Accuracy of regression model with Bag of words is {accuracy*100:.1f}%.")

Accuracy of regression model with Bag of words is 81.8%.


You should get about 88% accuracy, pretty good for such a simple model. Now let's do the same with a tf-idf encoding:

In [160]:
tfidf = TfidfVectorizer(
    max_features=max_vocab_size,
    )

x_train = tfidf.fit_transform(train_reviews)
x_test = tfidf.transform(test_reviews)

model2 = LogisticRegression(max_iter=10000)
model2.fit(x_train, train_sentiments)

# evaluate with accuracy
y_pred2 = model2.predict(x_test)

accuracy2 = accuracy_score(test_sentiments, y_pred2)
print(f"Accuracy of regression model is {accuracy2*100:.1f}%.")

Accuracy of regression model is 87.9%.


And you should get about 90% accuracy this time. Other classic but more sophisticated features include N-grams, part-of-speech tagging and syntax trees, you can read more about these there:

https://www.analyticsvidhya.com/blog/2020/07/part-of-speechpos-tagging-dependency-parsing-and-constituency-parsing-in-nlp/

But we will stop there for the classic approaches and go to deep learning methods.

...unless you are ahead of time, in this case learn about Bag of N-grams by yourself, and try them out :

In [ ]:
#TOFILL (Optional)


# Save data for LSTM

We have already covered feed-forward neural networks during the computer vision and the recommended system module. For natural language processing, one type of popular deep learning architecture is called Reccurent Neural Networks (RNNs). RNNs differ from feed-forward networks in the sense that some of their inner layers are recursively updated while iterating over the sequence of words given in input. We are going to use one specific RNN architecture called Long-Short Term Memory networks (LSTMs), which have been especially successful in various NLP tasks, including automatic translation, question answering, ... and text classification, our case study in this module.

Learn more about how RNNs and LSTMs encode texts as vectors first:

https://medium.com/towards-data-science/illustrated-guide-to-recurrent-neural-networks-79e5eb8049c9

If you want to understand in depth how one LSTM cell is working, you can go through these two articles:

https://medium.com/towards-data-science/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

http://colah.github.io/posts/2015-08-Understanding-LSTMs/


## Preprocessing

When using sequential models such as LSTMs, stopwords, punctuation and words suffixes carry semantics, and have thus much more importance than when using BOW-based models. Hence with these models we will only remove html tags, and keep all these :

In [14]:
imdb_deep_clean_dataset = normalize_text_dataset(
    imdb_dataset_original,
    html_tags = True,
    special_chars = False,
    lowercase = True,
    stemming = False, 
    stopwords = False,
    list_output = False
)

In [15]:
imdb_deep_clean_dataset.head(10)

,review,sentiment
0,i saw this movie last night after waiting ages...,positive
1,"this was, so far, the worst movie i have seen ...",negative
2,"in a time of magic, barbarians and demons abou...",positive
3,i had high expectations of this movie (the tit...,negative
4,this is a film of immense appeal to a relative...,negative
5,an hilariously accurate caricature of trying t...,positive
6,i watch most movies that nick mancuso is in be...,positive
7,this is one of those films that's more interes...,negative
8,a wonderful and gritty war film that focuses o...,positive
9,"man, some of you people have got to chill. thi...",positive


Keeping a validation set for early stopping is a good habit when training deep models, so let's resplit the dataset, and save the splits:

In [16]:
train_deep_clean = imdb_deep_clean_dataset.iloc[:20000]
valid_deep_clean = imdb_deep_clean_dataset.iloc[20000:25000]
test_deep_clean = imdb_deep_clean_dataset.iloc[25000:]

In [17]:
outdir = '../data/imdb_clean/'
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [18]:
train_deep_clean.to_csv(outdir + 'train.csv', index = False)
valid_deep_clean.to_csv(outdir + 'valid.csv', index = False)
test_deep_clean.to_csv(outdir + 'test.csv', index = False)